In [ ]:
####混合ガウス過程回帰モデル
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import gc
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [ ]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    if pattern==1:
        z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id

In [ ]:
####データの生成####
##データとセグメントの設定
#データの設定
seg = 8
T = 3000   
k = 100
Lambda = np.random.gamma(15.0, 1/0.5, T)
w = np.random.poisson(Lambda, T)
w[w <= 3] = 3

#セグメントの生成
pi = np.random.dirichlet(np.repeat(5.0, seg), 1).reshape(-1)
Z = np.random.multinomial(1, pi, T)
z = np.dot(Z, np.arange(seg))
index_z = [np.array(np.where(z==j)[0], dtype="int") for j in range(seg)] 
pit = pi.copy()

In [ ]:
##ガウス過程からデータの生成
#多項分布から入力変数を生成
alpha = np.repeat(0.15, k)
theta = np.random.dirichlet(alpha, seg)
data_freq = np.zeros((T, k), dtype="int")
data = np.zeros((T, k))
for i in range(T):
    data_freq[i, ] = np.random.multinomial(w[i], theta[z[i], ], 1)
    data[i, ] = data_freq[i, ] / w[i]
thetat = theta.copy()

#カーネル関数の生成
n = np.repeat(0, seg)
K = [j for j in range(seg)]
for j in range(seg):
    n[j] = index_z[j].shape[0]
    K[j] = np.dot(data[index_z[j], ], data[index_z[j], ].T)

In [ ]:
#モデルパラメータと応答変数を生成
y = np.repeat(0.0, T)
Sigma = np.power(0.1, 2)
beta = np.random.normal(0, 0.75, seg)
for j in range(seg):
    Sigma_diag = np.diag(np.repeat(Sigma, n[j]))
    y[index_z[j]] = beta[j] + np.random.multivariate_normal(np.repeat(0, n[j]), K[j] + Sigma_diag, 1).reshape(-1)
betat = beta.copy()

In [ ]:
####MCMCで混合ガウス過程回帰モデルを推定####
##パラメータ推定のための関数を定義
#多変量正規分布の条件付き期待値と分散を計算する関数
def cdMVN(mu, Cov, department, U):
    #分散共分散行列のブロック行列を定義
    department = np.array([department])
    index = np.delete(np.arange(Cov.shape[0]), department)
    Cov11 = Cov[department, ][:, department]
    Cov12 = Cov[department, ][:, index]
    Cov21 = Cov[:, department][index, ]
    Cov22 = Cov[index, ][:, index]

    #条件付き分散と条件付き平均を計算
    inv_Cov22 = np.linalg.inv(Cov22)
    CDinv = np.dot(Cov12, inv_Cov22)
    CDmu = mu[:, department] + np.dot(CDinv, (U[:, index] - mu[:, index]).T).T   #条件付き平均
    CDvar = Cov11 - np.dot(np.dot(Cov12, inv_Cov22), Cov21)   #条件付き分散
    return CDmu, CDvar

#多変量正規分布の密度関数
def mvdnorm(u, mu, Cov, k):
    er = U - mu 
    inv_Cov = np.linalg.inv(Cov)
    det_Cov = np.linalg.det(Cov)
    Lho = 1 / (np.power(np.sqrt(2*np.pi), k)*det_Cov) * np.exp(-1/2 * np.sum(np.dot(er, inv_Cov) * er, axis=1))
    return Lho 

In [ ]:
##アルゴリズムの設定
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 10

In [ ]:
#データの定数
par_freq = np.sum(data_freq, axis=0) / np.sum(data_freq)
const = scipy.special.loggamma(w + 1) - np.sum(scipy.special.loggamma(data_freq + 1), axis=1)

In [ ]:
##事前分布の設定
#潜在変数の事前分布
alpha1 = 1.0
alpha2 = 1.0

#モデルパラメータの事前分布
gamma = 0.0
tau = 100
eta = np.power(0.2, 2)
eta_sq = np.sqrt(eta)
Cov_diag = np.sum(data * data, axis=1) + eta

In [ ]:
##パラメータの真値
#潜在変数の真値
pi = pit.copy()
Zi = Z.copy()
z = np.dot(Zi, np.arange(seg))
n = np.sum(Zi, axis=0)
index_z = [j for j in range(seg)]
for j in range(seg):
    index_z[j] = np.hstack((np.array(np.where(z==j)[0], dtype="int")[:, np.newaxis], np.arange(np.sum(Zi[:, j]))[:, np.newaxis]))
    
#モデルパラメータの真値
theta = thetat.copy()
beta = betat.copy()
K = [j for j in range(seg)]
for j in range(seg):
    K[j] = np.dot(data[index_z[j][:, 0], ], data[index_z[j][:, 0], ].T)

In [ ]:
##パラメータの初期値
#潜在変数の初期値
pi = np.random.dirichlet(np.repeat(2.5, seg), 1).reshape(-1)
Zi = np.random.multinomial(1, pi, T)
z = np.dot(Zi, np.arange(seg))
n = np.sum(Zi, axis=0)
for j in range(seg):
    index_z[j] = np.hstack((np.array(np.where(z==j)[0], dtype="int")[:, np.newaxis], np.arange(np.sum(Zi[:, j]))[:, np.newaxis]))
    
#モデルパラメータの初期値
theta = np.random.dirichlet(np.repeat(3.0, k), seg)
beta = np.repeat(0.0, seg)
K = [j for j in range(seg)]
for j in range(seg):
    K[j] = np.dot(data[index_z[j][:, 0], ], data[index_z[j][:, 0], ].T)

In [ ]:
##パラメータの格納用配列
SEG = np.zeros((T, seg, int(R/keep)), dtype="int16")
PI = np.zeros((int(R/keep), seg))
THETA = np.zeros((seg, k, int(R/keep)))
BETA = np.zeros((int(R/keep), seg))

In [ ]:
##対数尤度の基準値
#1パラメータモデルの対数尤度
LLst = np.sum(scipy.stats.norm.logpdf(y, np.mean(y), np.sqrt(np.sum(np.power(y - np.mean(y), 2)) / T)))
print(LLst)

#真値での対数尤度

In [ ]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##セグメントパターン別の条件付き分布を定義
    #パラメータの格納用配列
    CDmu = np.zeros((T, seg))
    CDvar = np.zeros((T, seg))

    #条件付き分布を計算
    er = y - np.sum(beta * Zi, axis=1)   #モデル誤差
    for j in range(seg):
        index = index_z[j][:, 0]
        x = np.dot(data, data[index, ].T)
        u = er[index, ]
        inv_K = np.linalg.inv(K[j] + np.diag(np.repeat(eta, n[j])))
        CDmu[:, j] = np.dot(np.dot(x, inv_K), u)
        CDvar[:, j] = Cov_diag - np.sum(np.dot(x, inv_K) * x, axis=1)

    ##自身のレコードを除去した条件付き分布を定義
    for i in range(seg):
        #インデックスを抽出
        index_n = np.arange(n[i])
        index_z1 = index_z[i][:, 0]; index_z2 = index_z[i][:, 1]

        for j in range(index_n.shape[0]):
            #データの定義
            allocation = np.delete(index_n, index_z2[j])
            eta_diag = np.diag(np.repeat(eta, allocation.shape[0]))

            #条件付き分布と条件付き分布を計算
            x = np.dot(data[index_z1[j], ], data[index_z1[allocation], ].T)
            u = er[index_z1[allocation]]
            inv_K = np.linalg.inv(K[i][allocation, ][:, allocation] + eta_diag)
            CDmu[index_z1[j], i] = np.dot(np.dot(x, inv_K), u)
            CDvar[index_z1[j], i] = K[i][j, j] - np.dot(np.dot(x, inv_K), x)


    ##潜在変数zをサンプリング
    #セグメントパターン別の尤度
    beta_mu = np.sum(beta * Zi, axis=1)[:, np.newaxis]
    Lho1 = scipy.stats.norm.logpdf(y[:, np.newaxis], beta_mu + CDmu, np.sqrt(CDvar))
    Lho1 = np.exp(Lho1 - np.max(Lho1, axis=1)[:, np.newaxis])
    Lho2 = const[:, np.newaxis] + np.dot(data_freq, np.log(theta.T))
    Lho2 = np.exp(Lho2 - np.max(Lho2, axis=1)[:, np.newaxis])
    Lho = pi * Lho1 * Lho2

    #多項分布からzをサンプリング
    Prob = Lho / np.sum(Lho, axis=1)[:, np.newaxis]
    res = rmnom(Prob, T, seg, np.arange(T), 1)
    Zi = np.array(res[1].todense())
    z = np.array(res[0], dtype="int")
    index_z = [j for j in range(seg)]
    for j in range(seg):
        index_z[j] = np.hstack((np.array(np.where(z==j)[0], dtype="int")[:, np.newaxis], np.arange(np.sum(Zi[:, j]))[:, np.newaxis]))

    #混合率を更新
    pi = np.random.dirichlet(np.sum(Zi, axis=0) + alpha1, 1).reshape(-1)


    ##パラメータをサンプリング
    #カーネルを更新
    n = np.repeat(0, seg)
    K = [j for j in range(seg)]
    for j in range(seg):
        n[j] = index_z[j].shape[0]
        K[j] = np.dot(data[index_z[j][:, 0] , ], data[index_z[j][:, 0], ].T)

    #頻度パラメータをサンプリング
    seg_freq = np.dot(Zi.T, data_freq) + alpha2
    theta = np.zeros((seg, k))
    for j in range(seg):
        theta[j, ] = np.random.dirichlet(seg_freq[j, ], 1).reshape(-1)

    #期待値パラメータをサンプリング
    er = y - np.sum(Zi * CDmu, axis=1)
    mu = np.sum(er[:, np.newaxis] * Zi, axis=0) / n
    Sigma_sq = (np.sum(Zi * np.sqrt(CDvar), axis=0) / n) + eta_sq
    weights = tau / (Sigma_sq/n + tau)
    beta = np.random.normal(weights*mu, weights*Sigma_sq/n, seg)


    ##サンプリング結果の格納と表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        SEG[:, :, mkeep] = Zi
        PI[mkeep, ] = pi
        THETA[:, :, mkeep] = theta
        BETA[mkeep, ] = beta

    if rp%disp==0:
        #対数尤度の更新
        mu = np.sum(Zi * (beta + CDmu), axis=1)
        sigma = np.sum(Zi * np.sqrt(CDvar), axis=1) + eta_sq 
        LL = np.sum(scipy.stats.norm.logpdf(y, mu, sigma))

        #サンプリング結果の表示
        print(rp)
        print(np.round([LL, LLst], 1))
        print(np.round(np.hstack((beta, betat)).reshape(2, seg), 3))